<a href="https://colab.research.google.com/github/mmsamiei/world/blob/main/world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Faker

     |████████████████████████████████| 1.1MB 5.8MB/s 


In [2]:
from faker import Faker
import random
import datetime
from itertools import filterfalse
from dateutil import relativedelta


In [3]:
fake = Faker()

In [4]:
class Location:

  def __init__(self, map_size=50):
    self.x =  random.uniform(-map_size, map_size)
    self.y = random.uniform(-map_size, map_size)

  def distance(self, other_location):
    dx = other_location.x - self.x
    dy = other_location.y - self.y
    return (dx**2+dy**2)**0.5

In [5]:
random.choice([70,80,90,100])

90

In [31]:
class Person:
  
  def __init__(self, world):

    self.world = world
    self.gender = random.choices(['M', 'F'])
    self.first_name = fake.first_name_female() if self.gender == 'F' else fake.first_name_male()
    self.last_name = fake.last_name()
    self.birthday = self.world.date
    self.deathday = None
    self.location = Location()
    self.velocity = 10
    self.energy = 1
    self.hunger_thershold = random.choice([0.5,0.6,0.7,0.8,0.9,1])

  def __str__(self):
    x = round(self.location.x,2)
    y = round(self.location.y,2)
    age = relativedelta.relativedelta(self.world.date, self.birthday)

    return f"{self.first_name} {self.last_name}  at {x}, {y} energy: {self.energy}  age: {age.years, age.months}"
  

  def age(self):
    age = relativedelta.relativedelta(self.world.date, self.birthday)
    return age.years, age.months, age.days

  def choose_action(self):
    
    for food in self.world.foods:
      if self.location.distance(food.location) < self.world.distance_can_eat:
        return 'eat', (self, food)

    day_to_death = self.energy / self.world.hunger_penalty
    dx = 0
    dy = 0
    for food in random.sample(self.world.foods[:], len(self.world.foods[:])):
      if self.location.distance(food.location)/self.velocity < day_to_death:
        dx = food.location.x - self.location.x
        dy = food.location.y - self.location.y
        length = (dx**2+dy**2)**0.5
        dx_norm = (dx / length) * (self.velocity ** 0.5)
        dy_norm = (dy / length) * (self.velocity ** 0.5)
        if dx**2 + dy**2 < dx_norm**2 + dy_norm**2:
          dir_x, dir_y = dx, dy
        else:
          dir_x, dir_y= dx_norm, dy_norm
        break
    return 'move', (self, dx, dy)

In [32]:
class Food:

  def __init__(self, world):
    self.world = world
    self.location = Location()
    self.produce_date = self.world.date

In [33]:
class World:

  def __init__(self, name):
    self.name = name
    self.persons = []
    self.dead_persons = []
    self.date = datetime.date(2018, 6, 1)
    self.foods = []
    self.hunger_penalty = 0.1
    self.distance_can_eat = 2
    self.every_day_foods = 10

  def simulate_day(self):
    self.date += datetime.timedelta(days=1)

    new_person = Person(world=self)
    self.persons.append(new_person)

    for i in range(self.every_day_foods):
      self.foods.append(Food(self))
    self.foods[:] = filterfalse(lambda f: (world.date - f.produce_date).days > 5 , self.foods)

    for person in self.persons[:]:
      self.simulate_person(person)
  

  def simulate_person(self, person):
    person.energy -= self.hunger_penalty
    action, args = person.choose_action()
    
    if action == 'eat':
      food = args[1]
      self.foods.remove(food)
      person.energy = 1
    elif action == 'move':
      dx, dy = args[1], args[2]
      person.location.x += dx
      person.location.y += dy

    if person.energy <= 0 :
      self.persons.remove(person)
      self.dead_persons.append(person)
      person.deathday = self.date

  def simulate(self, until=365*10):
    for i in range(until):
      self.simulate_day()
      #print(len(self.foods))
      print("persons ",len(self.persons), len(self.dead_persons))


In [34]:
world = World('mahdi_world')
world.simulate()

persons  1 0
persons  2 0
persons  3 0
persons  4 0
persons  5 0
persons  6 0
persons  7 0
persons  8 0
persons  9 0
persons  10 0
persons  11 0
persons  12 0
persons  13 0
persons  14 0
persons  15 0
persons  16 0
persons  17 0
persons  18 0
persons  19 0
persons  20 0
persons  21 0
persons  22 0
persons  23 0
persons  24 0
persons  25 0
persons  26 0
persons  27 0
persons  28 0
persons  29 0
persons  30 0
persons  31 0
persons  32 0
persons  33 0
persons  34 0
persons  35 0
persons  36 0
persons  37 0
persons  38 0
persons  39 0
persons  40 0
persons  41 0
persons  42 0
persons  43 0
persons  44 0
persons  45 0
persons  45 1
persons  46 1
persons  46 2
persons  47 2
persons  46 4
persons  47 4
persons  47 5
persons  47 6
persons  48 6
persons  47 8
persons  47 9
persons  48 9
persons  49 9
persons  50 9
persons  50 10
persons  48 13
persons  47 15
persons  48 15
persons  48 16
persons  48 17
persons  48 18
persons  46 21
persons  46 22
persons  46 23
persons  46 24
persons  46 25
per

In [35]:
for person in world.persons:
  print(person.age())

(9, 11, 14)
(8, 11, 1)
(5, 6, 18)
(4, 11, 5)
(4, 7, 8)
(2, 9, 23)
(2, 3, 5)
(1, 11, 27)
(1, 7, 3)
(1, 5, 26)
(1, 4, 5)
(1, 3, 7)
(0, 7, 25)
(0, 7, 16)
(0, 7, 7)
(0, 5, 17)
(0, 5, 3)
(0, 4, 27)
(0, 3, 15)
(0, 3, 14)
(0, 3, 11)
(0, 3, 1)
(0, 2, 14)
(0, 2, 8)
(0, 1, 29)
(0, 1, 29)
(0, 1, 28)
(0, 1, 12)
(0, 1, 7)
(0, 1, 3)
(0, 0, 29)
(0, 0, 28)
(0, 0, 27)
(0, 0, 25)
(0, 0, 18)
(0, 0, 15)
(0, 0, 11)
(0, 0, 10)
(0, 0, 9)
(0, 0, 8)
(0, 0, 7)
(0, 0, 6)
(0, 0, 5)
(0, 0, 4)
(0, 0, 3)
(0, 0, 2)
(0, 0, 1)
(0, 0, 0)
